# Creating a DataBase 

In [5]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database = 'FBREF2'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'      
engine = create_engine(conn_str)
conn = engine.connect()
conn.execute("commit")
try:
    conn.execute(f'create database {database}')
except:
    print('database already exists')
conn.close() 

database already exists


In [3]:
import numpy as np

In [6]:
engine = create_engine(conn_str+f'/{database}')

In [6]:
attack = pd.read_csv('attack.csv')
defence = pd.read_csv('defence.csv')
gk = pd.read_csv('gk.csv')

# Replacing the columns with % as unable to put to SQL

In [7]:
gk.rename(columns = {gk.columns[-1]: 'team', gk.columns[-2]: 'game_id', gk.columns[8]: 'save_pct',gk.columns[12]: 'cmp_pct', gk.columns[15]: 'launch_pct', gk.columns[18]: 'launch_1pct', gk.columns[22]: 'stp_pct', gk.columns[23]: 'opa'}, inplace=True)

defence.rename(columns={defence.columns[14]: 'tkl_pct', defence.columns[18]: 'press_pct', defence.columns[-1]: 'team', defence.columns[-2]: 'game_id'}, inplace=True)

attack.rename(columns={attack.columns[27]: 'cmp_pct', attack.columns[-1]: 'team', attack.columns[-2]: 'game_id'}, inplace=True)


In [11]:
gk.drop(columns='Unnamed: 0', inplace=True)

attack.drop(columns='Unnamed: 0', inplace=True)


defence.drop(columns='Unnamed: 0', inplace=True)

In [26]:
attack.to_csv('att_f.csv')
gk.to_csv('gk_f.csv')
defence.to_csv('def_f.csv')


In [27]:
tables = !ls *.csv

In [28]:
tables

['att_f.csv', 'def_f.csv', 'gk_f.csv', 'meta.csv']

In [34]:
engine = create_engine(conn_str+f'/{database}')
for table in tables:
    table_name = table.split('.')[0]
    print(table_name)
    try:
        df = pd.read_csv(table)
        display(df.head(2))
        df.to_sql(table_name, engine, index=False, if_exists='replace')
    except:
        print('No values in ', (table))

att_f


,Unnamed: 0,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,...,Cmp,Att_1,cmp_pct,Prog_2,Carries,Prog_3,Succ,Att_4,game_id,team
0,0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1,0,0,...,19.0,24.0,79.2,1.0,24.0,2.0,1.0,1.0,1,Arsenal
1,1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,75.0,1,0,0,...,16.0,21.0,76.2,2.0,25.0,6.0,3.0,4.0,1,Arsenal


def_f


,Unnamed: 0,Player,#,Nation,Pos,Age,Min,Tkl_1,TklW,Def 3rd_2,...,Blocks,Sh,ShSv,Pass,Int,Tkl+Int,Clr,Err,game_id,team
0,0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1.0,1,0.0,...,1.0,0.0,0.0,1.0,0,1.0,0.0,0.0,1,Arsenal
1,1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,75.0,1.0,1,1.0,...,1.0,0.0,0.0,1.0,3,4.0,1.0,0.0,1,Arsenal


gk_f


,Unnamed: 0,Player,Nation,Age,Min,SoTA,GA,Saves,save_pct,PSxG,...,Att_5,launch_1pct,AvgLen_7,Opp,Stp,stp_pct,opa,AvgDist,game_id,team
0,0,Petr Čech,cz CZE,35-083,90.0,4.0,3.0,1.0,0.25,2.8,...,2.0,0.0,32.0,18.0,1.0,5.6,2.0,23.3,1,Arsenal
1,1,Kasper Schmeichel,dk DEN,30-279,90.0,10.0,4.0,6.0,0.60,3.1,...,11.0,100.0,75.5,12.0,0.0,0.0,1.0,20.0,1,Leicester City


meta


,Unnamed: 0,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG
0,0,"Friday August 11, 2017",Premier League,Manager: Arsène Wenger,Manager: Craig Shakespeare,Mike Dean (Referee),"Attendance: 59,387",Petr Čech,Arsenal,Leicester City,1,Wes Morgan,3,4,2.1,1.6
1,1,"Saturday August 12, 2017",Premier League,Manager: Marco Silva,Manager: Jürgen Klopp,Anthony Taylor (Referee),"Attendance: 20,407",Heurelho Gomes,Watford,Liverpool,2,Jordan Henderson,3,3,2.0,3.0


In [7]:
def q(query):
    x = pd.read_sql(query, engine)
    return x

### Attaching date to each row of dataframe

In [ ]:
gk_date = q('''SELECT * FROM gk_f a
        JOIN (SELECT "Date", "Game_id", "Competition" FROM meta) b
        ON a.game_id = b."Game_id"''')


In [390]:
gk_date.drop(columns=['Unnamed: 0', 'Game_id'], inplace=True)

In [395]:
gk_date[gk_date.duplicated([col for col in gk_date.columns if col != 'game_id'])].game_id.nunique()

598

In [400]:
gk_date[gk_date.duplicated([col for col in gk_date.columns if col != 'game_id'])]

,Player,Nation,Age,Min,SoTA,GA,Saves,save_pct,PSxG,Cmp,...,AvgLen_7,Opp,Stp,stp_pct,opa,AvgDist,game_id,team,Date,Competition
1915,Mathew Ryan,au AUS,27-264,90.0,2.0,0.0,2.0,1.0,0.4,13.0,...,58.6,4.0,0.0,0.0,0.0,10.7,950,Brighton & Hove Albion,"Saturday December 28, 2019",Premier League
1916,Mathew Ryan,au AUS,27-264,90.0,2.0,0.0,2.0,1.0,0.4,13.0,...,58.6,4.0,0.0,0.0,0.0,10.7,950,Brighton & Hove Albion,"Saturday December 28, 2019",Premier League
1918,Aaron Ramsdale,eng ENG,21-228,90.0,4.0,2.0,2.0,0.5,2.1,1.0,...,27.9,12.0,2.0,16.7,0.0,8.3,950,Bournemouth,"Saturday December 28, 2019",Premier League
1919,Aaron Ramsdale,eng ENG,21-228,90.0,4.0,2.0,2.0,0.5,2.1,1.0,...,27.9,12.0,2.0,16.7,0.0,8.3,950,Bournemouth,"Saturday December 28, 2019",Premier League
1921,Ben Foster,eng ENG,36-269,90.0,2.0,0.0,2.0,1.0,0.3,8.0,...,68.0,14.0,1.0,7.1,1.0,12.0,951,Watford,"Saturday December 28, 2019",Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16532,Matt Turner,us USA,26-165,90.0,7.0,1.0,6.0,85.7,1.2,13.0,...,46.4,8.0,0.0,0.0,1.0,13.6,9690,New England Revolution,"Sunday December 6, 2020",Major League Soccer
16533,Stefan Frei,ch SUI,34-231,90.0,5.0,2.0,3.0,60.0,1.1,6.0,...,44.0,5.0,0.0,0.0,0.0,10.0,9691,Seattle Sounders FC,"Monday December 7, 2020",Major League Soccer
16534,Dayne St. Clair,ca CAN,23-212,90.0,6.0,3.0,3.0,50.0,2.5,8.0,...,65.6,13.0,0.0,0.0,0.0,9.9,9691,Minnesota United,"Monday December 7, 2020",Major League Soccer
16535,Eloy Room,cw CUW,31-310,90.0,2.0,0.0,2.0,100.0,0.1,5.0,...,67.3,10.0,1.0,10.0,0.0,12.0,9692,Columbus Crew,"Saturday December 12, 2020",Major League Soccer


In [403]:
gk_date

,Player,Nation,Age,Min,SoTA,GA,Saves,save_pct,PSxG,Cmp,...,AvgLen_7,Opp,Stp,stp_pct,opa,AvgDist,game_id,team,Date,Competition
0,Petr Čech,cz CZE,35-083,90.0,4.0,3.0,1.0,0.25,2.8,1.0,...,32.0,18.0,1.0,5.6,2.0,23.3,1,Arsenal,"Friday August 11, 2017",Premier League
1,Kasper Schmeichel,dk DEN,30-279,90.0,10.0,4.0,6.0,0.60,3.1,10.0,...,75.5,12.0,0.0,0.0,1.0,20.0,1,Leicester City,"Friday August 11, 2017",Premier League
2,Heurelho Gomes,br BRA,36-178,90.0,4.0,3.0,2.0,0.50,2.6,12.0,...,75.1,8.0,0.0,0.0,2.0,16.6,2,Watford,"Saturday August 12, 2017",Premier League
3,Simon Mignolet,be BEL,29-159,90.0,5.0,3.0,2.0,0.40,2.5,2.0,...,51.0,12.0,2.0,16.7,0.0,13.7,2,Liverpool,"Saturday August 12, 2017",Premier League
4,Ben Foster,eng ENG,34-131,90.0,2.0,0.0,2.0,1.00,0.4,5.0,...,76.5,18.0,0.0,0.0,0.0,13.7,3,West Bromwich Albion,"Saturday August 12, 2017",Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,Brian Rowe,us USA,30-222,90.0,4.0,1.0,3.0,75.00,0.2,7.0,...,53.9,4.0,1.0,25.0,1.0,13.5,9694,Orlando City,"Wednesday June 26, 2019",Major League Soccer
16541,Quentin Westberg,us USA,33-062,90.0,3.0,2.0,2.0,66.70,1.3,8.0,...,28.6,14.0,0.0,0.0,0.0,10.4,9695,Toronto FC,"Wednesday June 26, 2019",Major League Soccer
16542,Brad Guzan,us USA,34-290,90.0,3.0,3.0,1.0,33.30,2.4,2.0,...,28.7,10.0,1.0,10.0,1.0,17.4,9695,Atlanta United,"Wednesday June 26, 2019",Major League Soccer
16543,Evan Bush,us USA,33-112,90.0,3.0,1.0,2.0,66.70,1.1,5.0,...,54.2,2.0,1.0,50.0,0.0,8.0,9696,Montreal Impact,"Wednesday June 26, 2019",Major League Soccer


# Dropping Duplicates

In [407]:
gk_no_d = gk_date.drop_duplicates([col for col in gk_date.columns if col != 'game_id'])

In [408]:
gk_no_d.game_id.nunique()

7591

In [384]:
meta_no_d[meta_no_d['Game_id']==951]

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG,949
949,"Saturday December 28, 2019",Premier League,Manager: Nigel Pearson,Manager: Dean Smith,Simon Hooper (Referee),"Attendance: 21,348",Troy Deeney,Watford,Aston Villa,951.0,Jack Grealish,0.0,3,2.9,0.5,NaN


In [383]:
meta_no_d.loc[949, 'Game_id'] = 951

In [262]:
meta[(meta['Home_team']=='Strasbourg') & (meta['Away_team']=='Lille')]

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG
1488,"Sunday August 13, 2017",Ligue 1,Manager: Thierry Laurey,Manager: Marcelo Bielsa,Johan Hamel (Referee),"Attendance: 25,591",Kader Mangane,Strasbourg,Lille,2734,Ibrahim Amadou,0,3,3.4,0.8
2102,"Friday February 22, 2019",Ligue 1,Manager: Thierry Laurey,Manager: Christophe Galtier,Clément Turpin (Referee),"Attendance: 25,303",Pablo Martinez,Strasbourg,Lille,3347,José Fonte,1,1,0.6,1.2
2442,"Saturday February 1, 2020",Ligue 1,Manager: Thierry Laurey,Manager: Christophe Galtier,Stéphanie Frappart (Referee),"Attendance: 25,139",Stefan Mitrović,Strasbourg,Lille,3687,José Fonte,2,1,1.6,1.3
2559,"Sunday October 4, 2020",Ligue 1,Manager: Thierry Laurey,Manager: Christophe Galtier,Mikael Lesage (Referee),"Venue: Stade de la Meinau, Strasbourg",Stefan Mitrović,Strasbourg,Lille,3804,José Fonte,3,0,1.7,1.9


In [227]:
meta.drop(columns= 'Unnamed: 0', inplace=True)

In [152]:
att_date[(att_date.game_id >1600) & (att_date.game_id < 3987)].sort_values(['game_id', 'team']).game_id.nunique()

1269

# Loading in my Dataframes

In [8]:
gk = q('''SELECT * FROM gk_f''')
attack = q('''SELECT * FROM att_f''')
defence = q('''SELECT * FROM def_f''')
meta = q('''SELECT * FROM meta''')

In [8]:
q('''SELECT * FROM def_f''')

,Player,#,Nation,Pos,Age,Min,Tkl_1,TklW,Def 3rd_2,Mid 3rd_3,...,ShSv,Pass,Int,Tkl+Int,Clr,Err,game_id,team,Date,Competition
0,Alexandre Lacazette,9.0,fr FRA,"FW,LW",26-075,90.0,1.0,1,0.0,0.0,...,0.0,1.0,0,1.0,0.0,0.0,1,Arsenal,"Friday August 11, 2017",Premier League
1,Danny Welbeck,23.0,eng ENG,"AM,RW",26-258,75.0,1.0,1,1.0,0.0,...,0.0,1.0,3,4.0,1.0,0.0,1,Arsenal,"Friday August 11, 2017",Premier League
2,Theo Walcott,14.0,eng ENG,RW,28-148,15.0,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,1.0,0.0,1,Arsenal,"Friday August 11, 2017",Premier League
3,Mesut Özil,11.0,de GER,AM,28-300,90.0,1.0,0,1.0,0.0,...,0.0,2.0,0,1.0,0.0,0.0,1,Arsenal,"Friday August 11, 2017",Premier League
4,Granit Xhaka,29.0,ch SUI,DM,24-318,90.0,1.0,0,0.0,1.0,...,0.0,0.0,0,1.0,1.0,0.0,1,Arsenal,"Friday August 11, 2017",Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229715,Claude Dielna,5.0,GPE,CB,31-194,90.0,2.0,2,2.0,0.0,...,0.0,0.0,3,5.0,11.0,0.0,9696,Portland Timbers,"Wednesday June 26, 2019",Major League Soccer
229716,Julio Cascante,18.0,cr CRC,CB,25-266,90.0,0.0,0,0.0,0.0,...,0.0,1.0,0,0.0,3.0,0.0,9696,Portland Timbers,"Wednesday June 26, 2019",Major League Soccer
229717,Modou Jadama,94.0,us USA,RB,25-101,90.0,5.0,2,1.0,4.0,...,0.0,1.0,2,7.0,1.0,0.0,9696,Portland Timbers,"Wednesday June 26, 2019",Major League Soccer
229718,Jeff Attinella,1.0,us USA,GK,30-270,90.0,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,9696,Portland Timbers,"Wednesday June 26, 2019",Major League Soccer


# Dropping Duplicates

In [336]:
att_no_d = att.drop_duplicates([col for col in att.columns if col !='game_id'])

In [337]:
att_no_d.game_id.nunique()

7591

In [279]:
meta[meta.duplicated([col for col in meta.columns if col !='Game_id'])].Game_id.nunique()

431

In [290]:
def_no_d = defence.drop_duplicates([col for col in defence.columns if col !='Game_id'])

In [416]:
def_date.drop(columns='Game_id', inplace=True)

In [418]:
def_no_d = def_date.drop_duplicates(subset=[col for col in def_date.columns if col != 'game_id'])

In [326]:
defence.drop(columns='Game_id', inplace=True)

In [331]:
defence[defence.duplicated([col for col in defence.columns if col !='game_id'])].game_id.nunique()

432

In [285]:
meta_no_d = meta.drop_duplicates([col for col in meta.columns if col !='Game_id'])

In [300]:
meta_no_d[meta_no_d.duplicated()]

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG


In [302]:
big_meta = meta_no_d

In [304]:
attacks = att_no_d

In [305]:
defences = def_no_d

In [385]:
meta_no_d.Game_id.nunique()

7591

In [311]:
meta.Game_id.nunique()

7915

In [321]:
len(meta.columns)

15

In [340]:
meta_no_d_game_id = meta_no_d[['Game_id']]

In [341]:
meta_no_d_game_id

,Game_id
0,1
1,2
2,3
3,4
4,5
...,...
7695,9368
8020,9693
8021,9694
8022,9695


In [348]:
att_game_id = att_no_d.groupby('game_id').mean()

In [357]:
game_ids = att_game_id.index

In [360]:
meta_ids = meta_no_d_game_id.Game_id

In [362]:
m_ids = []
for i in meta_ids:
    m_ids.append(i)

In [365]:
a_ids = []
for i in game_ids:
    a_ids.append(i)

In [410]:
meta_no_d.Game_id.nunique()

7591

In [411]:
att_no_d.game_id.nunique()

7591

In [419]:
def_no_d.game_id.nunique()

7591

In [420]:
gk_no_d.game_id.nunique()

7591

# Saving Changes to DataBase

In [423]:
gk_f.to_sql('gk_f', engine, index=False, if_exists='replace')
def_f.to_sql('def_f', engine, index=False, if_exists='replace')
att_f.to_sql('att_f', engine, index=False, if_exists='replace')
meta.to_sql('meta', engine, index=False, if_exists='replace')

In [58]:
gk['Date'] = pd.to_datetime(gk.Date)

defence['Date'] = pd.to_datetime(defence.Date)

attack['Date'] = pd.to_datetime(attack.Date)

# meta['Date'] = pd.to_datetime(meta.Date)

In [63]:
test['Player'] = test.Player.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

# Cleaning Data

In [48]:
meta.home_captain.fillna('John Egan', inplace=True)

In [ ]:
meta.away_captain.fillna('')

In [458]:
meta.Attendance.apply(lambda x: x.str.replace.contains('Venue'))

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG,949
6551,2018-03-03,Major League Soccer,Manager: Greg Vanney,Manager: Gregg Berhalter,Ismail Elfath (Referee),"Attendance: 26,633",Michael Bradley,Toronto FC,Columbus Crew,8223.0,Wil Trapp,2.0,0,1.2,1.1,NaN
6552,2018-03-03,Major League Soccer,Manager: Wílmer Cabrera,Manager: Gerardo Martino,Allen Chapman (Referee),"Attendance: 20,377",DaMarcus Beasley,Houston Dynamo,Atlanta United,8224.0,Jeff Larentowicz,0.0,4,2.6,1.9,NaN
6553,2018-03-03,Major League Soccer,Manager: Jim Curtin,Manager: Brad Friedel,Rubiel Vazquez (Referee),"Attendance: 16,452",Alejandro Bedoya,Philadelphia Union,New England Revolution,8225.0,Claude Dielna,0.0,2,3.6,0.5,NaN
6554,2018-03-03,Major League Soccer,Manager: Jason Kreis,Manager: Ben Olsen,Fotis Bazakos (Referee),"Attendance: 25,527",Jonathan Spector,Orlando City,D.C. United,8226.0,Steve Birnbaum,1.0,1,0.9,2.1,NaN
6555,2018-03-03,Major League Soccer,Manager: Óscar Pareja,Manager: Mike Petke,Robert Sibiga (Referee),"Attendance: 16,116",Matt Hedges,FC Dallas,Real Salt Lake,8227.0,Kyle Beckerman,1.0,1,1.3,0.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7691,2020-12-01,Major League Soccer,Manager: Brian Schmetzer,Manager: Luchi González,Robert Sibiga (Referee),"Venue: CenturyLink Field, Seattle, WA",Nicolás Lodeiro,Seattle Sounders FC,FC Dallas,9364.0,Matt Hedges,0.0,1,1.1,0.7,NaN
7692,2020-12-03,Major League Soccer,Manager: Peter Vermes,Manager: Adrian Heath,Drew Fischer (Referee),"Venue: Children's Mercy Park, Kansas City, MO",Johnny Russell,Sporting KC,Minnesota United,9365.0,Osvaldo Alonso,3.0,0,1.6,1.4,NaN
7693,2020-12-06,Major League Soccer,Manager: Caleb Porter,Manager: Bruce Arena,Armando Villarreal (Referee),"Venue: MAPFRE Stadium, Columbus, OH",Jonathan Mensah,Columbus Crew,New England Revolution,9366.0,Carles Gil,0.0,1,1.4,1.0,NaN
7694,2020-12-07,Major League Soccer,Manager: Brian Schmetzer,Manager: Adrian Heath,Ismail Elfath (Referee),"Venue: CenturyLink Field, Seattle, WA",Nicolás Lodeiro,Seattle Sounders FC,Minnesota United,9367.0,Osvaldo Alonso,2.0,3,1.8,0.6,NaN


# Meta Cleaning

In [100]:
meta

,Date,Competition,Home_manager,Away_manager,Ref,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG
0,2017-08-11,Premier League,Manager: Arsène Wenger,Manager: Craig Shakespeare,Mike Dean (Referee),"Attendance: 59,387",Petr Čech,Arsenal,Leicester City,1.0,Wes Morgan,3.0,4.0,2.1,1.6
1,2017-08-12,Premier League,Manager: Marco Silva,Manager: Jürgen Klopp,Anthony Taylor (Referee),"Attendance: 20,407",Heurelho Gomes,Watford,Liverpool,2.0,Jordan Henderson,3.0,3.0,2.0,3.0
2,2017-08-12,Premier League,Manager: Tony Pulis,Manager: Eddie Howe,Robert Madley (Referee),"Attendance: 25,011",Jake Livermore,West Bromwich Albion,Bournemouth,3.0,Simon Francis,0.0,1.0,1.2,0.4
3,2017-08-12,Premier League,Manager: Ronald Koeman,Manager: Mark Hughes,Niel Swarbrick (Referee),"Attendance: 39,045",Phil Jagielka,Everton,Stoke City,4.0,Ryan Shawcross,0.0,1.0,0.4,0.3
4,2017-08-12,Premier League,Manager: Mauricio Pellegrino,Manager: Paul Clement,Mike Jones (Referee),"Attendance: 31,447",Steven Davis,Southampton,Swansea City,5.0,Leon Britton,0.0,0.0,2.1,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7586,2020-12-12,Major League Soccer,Manager: Caleb Porter,Manager: Brian Schmetzer,Jair Marrufo (Referee),"Attendance: 1,500",Jonathan Mensah,Columbus Crew,Seattle Sounders FC,9368.0,Nicolás Lodeiro,0.0,3.0,1.6,0.5
7587,2019-06-26,Major League Soccer,Manager: Luchi González,Manager: Marc Dos Santos,Silviu Petrescu (Referee),"Attendance: 11,911",Reto Ziegler,FC Dallas,Vancouver Whitecaps FC,9693.0,Jon Erice,2.0,2.0,2.7,0.4
7588,2019-06-26,Major League Soccer,Manager: Ben Olsen,Manager: James O'Connor,Alex Chilowicz (Referee),"Attendance: 17,398",Wayne Rooney,D.C. United,Orlando City,9694.0,Nani,0.0,1.0,0.7,1.4
7589,2019-06-26,Major League Soccer,Manager: Greg Vanney,Manager: Frank de Boer,Alan Kelly (Referee),"Attendance: 24,598",Quentin Westberg,Toronto FC,Atlanta United,9695.0,Michael Parkhurst,2.0,3.0,2.1,2.5


In [86]:
import matplotlib.pyplot as plt

In [93]:
meta['away_conceded'] = meta.away_conceded.apply(lambda x: x.replace('*', ''))

In [95]:
def convert_to_float(x):
    try:
        return float(x)
    except:
        print(f'{x} cannot be converted to float')
        return np.nan

In [97]:
meta['away_conceded'] = meta.away_conceded.apply(convert_to_float)

#### Removing accents from name columns

In [ ]:
test['Player'] = test.Player.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [101]:
meta['home_captain'] = meta.home_captain.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [102]:
meta['away_captain'] = meta.away_captain.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [428]:
meta['Home_manager'] = meta.Home_manager.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [429]:
meta['Away_manager'] = meta.Away_manager.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [431]:
meta['Home_team'] = meta.Home_team.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [432]:
meta['Away_team'] = meta.Away_team.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [107]:
meta.away_captain.fillna('Federico Fernandez', inplace=True)

In [113]:
meta['Home_manager'] = meta.Home_manager.apply(lambda x: x.replace('Manager: ', ''))

In [116]:
meta['Away_manager'] = meta.Away_manager.apply(lambda x: x.replace('Manager: ', ''))

In [119]:
meta['Ref'] = meta.Ref.apply(lambda x: x.replace('(Referee)', ''))

In [123]:
meta['Attendance'] = meta.Attendance.apply(lambda x: x.replace('Attendance: ', ''))

In [284]:
meta.loc[meta['Attendance'].str.contains('Venue', case=False), 'Attendance']= '0'

In [285]:
meta['Attendance'] = meta.Attendance.apply(lambda x: x.replace(',', ''))

In [287]:
meta['Attendance'] = meta.Attendance.apply(convert_to_float)

  cannot be converted to float


In [378]:
meta.Attendance.fillna(0, inplace=True)

In [98]:
attack = q('''SELECT * FROM att_f''')

# Attack Cleaning 

#### Getting rid of accents in player and team names

In [99]:
attack['Player'] = attack.Player.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [100]:
attack['team'] = attack.team.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

#### Removing the summary row of each team

In [101]:
attack = attack.drop(attack[attack.Player.str.contains('Players')].index, axis=0)

#### Removing all goalkeepers from dataframe as it has its own seperate dataframe

In [102]:
attack = attack.drop(attack[attack.Pos.str.contains('GK') & (attack['Player']!='Leandro Cabrera')].index, axis=0)

#### Removing these players as all their values are NaN

In [103]:
# Can delete

attack = attack.drop(attack[attack['Player']=='Jordan Thomas'].index, axis=0)
attack = attack.drop(attack[attack['Player']=='Andrea Ghezzi'].index, axis=0)
attack = attack.drop(attack[attack.Player == 'Sergio Moreno'].index, axis=0)

In [496]:
attack_t = att_test.copy()

# Defence Cleaning

In [106]:
defence = q('SELECT * FROM def_f')

#### Getting rid of accents in player and team names

In [107]:
defence['Player'] = defence.Player.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [108]:
defence['team'] = defence.team.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

#### Removing all goalkeepers from dataframe as it has its own seperate dataframe

In [109]:
defence = defence.drop(defence[defence.Pos.str.contains('GK') & (defence['Player']!='Leandro Cabrera')].index, axis=0)

#### Removing the summary row of each team

In [110]:
defence = defence.drop(defence[defence.Player.str.contains('Players')].index, axis=0)

#### Removing these players as all their values are NaN

In [112]:
defence = defence.drop(defence[defence['Player']=='Jordan Thomas'].index, axis=0)
defence = defence.drop(defence[defence['Player']=='Andrea Ghezzi'].index, axis=0)
defence = defence.drop(defence[defence.Player == 'Sergio Moreno'].index, axis=0)

# GK Cleaning

In [113]:
gk = q('SELECT * FROM gk_f')

#### Getting rid of accents in player and team names

In [114]:
gk['Player'] = gk.Player.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [115]:
gk['team'] = gk.team.str.normalize('NFD')\
                .str.encode('ascii', errors='ignore')\
               .str.decode('utf-8')

In [848]:
gk_test = gk.copy()

#### Making sure that the save percentage is consistent as some were as decimal some were as percentage

In [119]:
saves = []

for x in gk['save_pct']:
    if x < 1:
        saves.append(x*100)
    else:
        saves.append(x)

In [859]:
gk_test['save_pct'] = saves

In [120]:
gk['save_pct'] = saves

In [963]:
gk_teams = gk.groupby(['game_id', 'team'])

In [121]:
gk_variables = list(gk.iloc[:, 3:-4].columns)

In [1613]:
# gk_teams.head(2).sort_values(by='Min')

# Sorting DF into exponentially weighted means and concatenating 

### Here I am changing the ages to floats and positions to one position rather than multiple

In [124]:
ages1 = []
for age in attack.Age:
    ages1.append(round((int(age.split('-')[0]) + (int(age.split('-')[1])/365)),3))

In [126]:
new_pos1 = []
for pos in attack.Pos:
    new_pos1.append(pos.split(',')[0])
# attack['Pos'] = new_pos1

new_pos = []
for pos in defence.Pos:
    new_pos.append(pos.split(',')[0])
# defence['Pos'] = new_pos

ages1 = []
for age in attack.Age:
    try:
        ages1.append(round((int(age.split('-')[0]) + (int(age.split('-')[1])/365)),3))
    except:
        ages1.append(np.nan)
# attack['Age'] = ages1

ages = []
for age in defence.Age:
    try:
        ages.append(round((int(age.split('-')[0]) + (int(age.split('-')[1])/365)),3))
    except:
        ages.append(np.nan)
# defence['Age'] = ages

ages2 = []
for age in gk.Age:
    try:
        ages2.append(round((int(age.split('-')[0]) + (int(age.split('-')[1])/365)),3))
    except:
        ages2.append(np.nan)


In [128]:
attack['Pos'] = new_pos1

In [130]:
defence['Pos'] = new_pos

In [132]:
attack['Age'] = ages1

In [134]:
defence['Age'] = ages

In [136]:
gk['Age'] = ages2

#### Assigning variables to attach back onto the EWMs for each player

In [137]:
variables = list(attack.iloc[:, 5:-4].columns)

In [138]:
defence_variables = list(defence.iloc[:, 5:-4].columns)

In [139]:
gk_variables = list(gk.iloc[:, 3:-4])

In [176]:
pres = ['Player', 'Pos','team', 'game_id', 'Age']

In [141]:
attack['Date'] = pd.to_datetime(attack.Date)

In [142]:
attack.set_index('Date', inplace=True)

In [70]:
test_frames = attack.copy()
test = test_frames.groupby(
    'Player').get_group(
    test_frames['Player'].unique()[134])
test.set_index("Date", inplace=True)
test_ewm = test[variables]
test_ewm.columns = ['{}_{}'.format('ewm', col) for col in test_ewm.columns]
all_columns = list(attack.columns) + list(test_ewm.columns)
print('TEST :\n')
display(test_ewm.ewm(span=12, min_periods=2).mean())

TEST :



,ewm_Min,ewm_Gls,ewm_Ast,ewm_PK,ewm_PKatt,ewm_Sh,ewm_SoT,ewm_CrdY,ewm_CrdR,ewm_Touches,...,ewm_SCA,ewm_GCA,ewm_Cmp,ewm_Att_1,ewm_cmp_pct,ewm_Prog_2,ewm_Carries,ewm_Prog_3,ewm_Succ,ewm_Att_4
Date,,,,,,,,,,,,,,,,,,,,,
2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-20,83.500000,0.000000,0.458333,0.000000,0.000000,0.541667,0.000000,0.000000,0.0,55.083333,...,4.458333,0.458333,29.708333,43.916667,67.158333,3.000000,39.166667,8.208333,1.916667,3.916667
2017-08-27,86.036952,0.000000,0.279446,0.000000,0.000000,1.501155,0.390300,0.000000,0.0,61.685912,...,4.669746,0.279446,37.237875,50.193995,72.833949,4.561201,45.736721,10.859122,2.339492,4.339492
2017-09-09,67.401293,0.000000,0.191236,0.000000,0.000000,1.658621,0.582759,0.000000,0.0,48.211638,...,3.827011,0.191236,28.955603,38.137644,78.789224,3.752730,35.403017,7.746983,1.916667,3.601006
2017-09-17,68.107352,0.000000,0.139278,0.000000,0.000000,1.751372,0.696122,0.000000,0.0,44.622083,...,3.058923,0.139278,25.435612,33.753104,77.134804,3.548216,33.391644,6.728941,1.667612,3.437715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-02,67.385179,0.037031,0.147850,0.023765,0.023765,1.115707,0.286553,0.010898,0.0,48.348349,...,3.184336,0.246177,31.930651,41.378824,78.130792,2.726766,33.769306,6.506160,0.598018,0.799087
2021-01-18,58.864382,0.031334,0.125104,0.020109,0.020109,1.097906,0.242468,0.009221,0.0,42.756295,...,2.848284,0.208303,28.402859,36.551312,79.956824,2.307264,29.958644,5.659059,0.506015,0.676151
2021-01-26,52.731400,0.026514,0.105857,0.017015,0.017015,0.928997,0.205165,0.007802,0.0,37.716865,...,2.410086,0.176257,24.802419,31.851110,80.471159,1.952300,25.811160,4.788434,0.428167,0.572128


In [76]:
from tqdm import tqdm

In [143]:
attack2 = attack.sort_values('Date')

In [144]:
attack2

,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Att_1,cmp_pct,Prog_2,Carries,Prog_3,Succ,Att_4,game_id,team,Competition
Date,,,,,,,,,,,,,,,,,,,,,
2017-08-04,Kylian Mbappe,10.0,fr FRA,FW,18.622,75.0,0,0,0,0,...,31.0,61.3,0.0,44.0,9.0,4.0,7.0,2717,Monaco,Ligue 1
2017-08-04,Kelvin Amian,2.0,fr FRA,RB,19.485,90.0,0,0,0,0,...,21.0,85.7,1.0,7.0,1.0,0.0,0.0,2717,Toulouse,Ligue 1
2017-08-04,Steeve Yago,20.0,bf BFA,CB,24.633,90.0,0,0,0,0,...,7.0,85.7,1.0,4.0,0.0,0.0,0.0,2717,Toulouse,Ligue 1
2017-08-04,Christopher Jullien,6.0,fr FRA,CB,24.370,90.0,0,0,0,0,...,13.0,61.5,2.0,3.0,1.0,0.0,0.0,2717,Toulouse,Ligue 1
2017-08-04,Francois Moubandje,29.0,ch SUI,LB,27.121,90.0,0,0,0,0,...,24.0,41.7,0.0,15.0,0.0,1.0,1.0,2717,Toulouse,Ligue 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-09,Carles Alena,11.0,es ESP,FW,23.096,36.0,0,0,0,0,...,19.0,84.2,3.0,15.0,2.0,0.0,0.0,8220,Getafe,La Liga
2021-02-09,Mathias Olivera,17.0,uy URU,LB,23.277,90.0,0,0,0,0,...,41.0,70.7,1.0,29.0,4.0,0.0,1.0,8220,Getafe,La Liga
2021-02-09,Erick Cabaco,3.0,uy URU,CB,25.811,72.0,0,0,0,0,...,16.0,56.3,2.0,10.0,0.0,0.0,1.0,8220,Getafe,La Liga


In [290]:
attack_ewm_dfs2 = []
for player in tqdm(attack2['Player'].unique()):
    a_box = np.array(
        attack2[attack2['Player']==player][pres])
    b_box = np.array(
        attack2[attack2['Player']==player][variables].ewm(
            span=12, min_periods=1).mean().shift())
    ab_box = np.concatenate([a_box, b_box], axis=1)
    attack_ewm_dfs2.append(ab_box)

100%|██████████| 5020/5020 [03:58<00:00, 21.05it/s]


In [178]:
att_columns = pres+ variables

In [291]:
tester = pd.DataFrame(attack_ewm_dfs2[10], columns=att_columns)
tester

,Player,Pos,team,game_id,Age,Min,Gls,Ast,PK,PKatt,...,SCA,GCA,Cmp,Att_1,cmp_pct,Prog_2,Carries,Prog_3,Succ,Att_4
0,Corentin Jean,AM,Toulouse,2717,22.055,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Corentin Jean,FW,Toulouse,2731,22.077,90,0,0,0,0,...,1,1,8,11,72.7,0,13,1,0,1
2,Corentin Jean,LW,Toulouse,2745,22.099,54.25,0,0,0,0,...,1,0.458333,5.83333,8.83333,64.25,0,9.20833,0.458333,0,0.458333
3,Corentin Jean,FW,Toulouse,2748,22.115,50.6397,0,0,0,0,...,1,0.669746,6.67898,10.0693,65.2062,0.3903,10.2979,1.06005,0,0.279446
4,Corentin Jean,RM,Toulouse,2761,22.153,41.2836,0,0,0,0,...,1,0.773994,6.46466,9.41609,68.2977,0.582759,8.94124,1.04109,0,0.506897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Corentin Jean,RW,Lens,3862,25.375,31.3536,0.00102466,0.130417,0,0,...,1.16208,0.194841,7.35313,10.5407,77.0044,0.306652,10.7101,1.66555,0.442017,0.583644
63,Corentin Jean,DM,Lens,3893,25.422,30.9915,0.000867017,0.110353,0,0,...,1.13714,0.164865,7.45265,10.3037,78.8348,0.259473,10.6009,1.87086,0.374013,0.49385
64,Corentin Jean,AM,Lens,3898,25.43,28.0693,0.000733627,0.0933749,0,0,...,0.962194,0.1395,7.38301,10.1031,78.6756,0.219553,10.2007,1.58303,0.316471,0.417872
65,Corentin Jean,WB,Lens,3937,25.488,26.3661,0.000620759,0.0790092,0,0,...,0.96801,0.118038,7.47793,10.395,76.8329,0.339624,10.3237,1.33948,0.421631,0.507432


In [181]:
defence2 = defence.sort_values('Date')

In [182]:
def_columns = pres + defence_variables

In [292]:
defence_ewm_dfs2 = []
for player in tqdm(defence2['Player'].unique()):
    a_box = np.array(
        defence2[defence2['Player']==player][pres])
    b_box = np.array(
        defence2[defence2['Player']==player][defence_variables].ewm(
            span=12, min_periods=1).mean().shift())
    ab_box = np.concatenate([a_box, b_box], axis=1)
    defence_ewm_dfs2.append(ab_box)

100%|██████████| 5020/5020 [03:46<00:00, 22.16it/s]


In [293]:
tester =pd.DataFrame(defence_ewm_dfs2[8], columns=def_columns)
tester

,Player,Pos,team,game_id,Age,Min,Tkl_1,TklW,Def 3rd_2,Mid 3rd_3,...,Mid 3rd_7,Att 3rd_8,Blocks,Sh,ShSv,Pass,Int,Tkl+Int,Clr,Err
0,Yann Bodiger,CM,Toulouse,2717,22.482,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yann Bodiger,DM,Toulouse,2731,22.504,26,0,0,0,0,...,2,0,0,0,0,0,1,1,0,0
2,Yann Bodiger,DM,Toulouse,2766,22.597,60.6667,1.625,0.541667,0,1.625,...,9.04167,0.541667,1.08333,0,0,1.08333,1,2.625,1.08333,0
3,Yann Bodiger,DM,Toulouse,2790,22.638,38.94,0.990762,0.330254,0,0.990762,...,6.2933,0.330254,0.660508,0,0,0.660508,0.6097,1.60046,0.660508,0
4,Yann Bodiger,FW,Toulouse,2851,22.792,27.9108,0.678017,0.226006,0,0.678017,...,4.30675,0.226006,0.452011,0,0,0.452011,0.417241,1.09526,0.452011,0
5,Yann Bodiger,CM,Toulouse,2863,22.803,28.4784,0.493802,0.164601,0,0.493802,...,5.58189,0.97969,0.329202,0,0,0.329202,0.575575,1.06938,0.329202,0
6,Yann Bodiger,LW,Toulouse,3074,23.236,35.4107,0.373782,0.124594,0,0.373782,...,4.95436,1.47073,0.249188,0,0,0.249188,0.678732,1.05251,0.249188,0
7,Yann Bodiger,AM,Toulouse,3190,23.693,29.9635,0.290374,0.0967914,0,0.290374,...,4.2951,1.36569,0.193583,0,0,0.193583,0.527276,0.81765,0.193583,0
8,Yann Bodiger,AM,Cadiz,8007,25.592,26.4235,0.229778,0.0765925,0,0.229778,...,3.39878,1.08069,0.153185,0,0,0.153185,0.417241,0.647019,0.153185,0
9,Yann Bodiger,DM,Cadiz,8015,25.614,30.0986,0.382155,0.0614398,0,0.382155,...,4.5069,1.4604,0.320715,0,0,0.320715,0.334696,0.716851,0.320715,0


In [153]:
gk['Date'] = pd.to_datetime(gk.Date)
gk.set_index('Date', inplace=True)

In [154]:
gk2 = gk.sort_values('Date')

In [155]:
gk_pres = ['team', 'game_id', 'Player', 'Age']

In [295]:
gk_ewm_dfs = []
for player in tqdm(gk2['Player'].unique()):
    a_box = np.array(
        gk2[gk2['Player']==player][gk_pres])
    b_box = np.array(
        gk2[gk2['Player']==player][gk_variables].ewm(
            span=12, min_periods=1).mean().shift())
    ab_box = np.concatenate([a_box, b_box], axis=1)
    gk_ewm_dfs.append(ab_box)


100%|██████████| 432/432 [00:05<00:00, 74.28it/s]


In [157]:
gk_columns = gk_pres + gk_variables

In [296]:
tester = pd.DataFrame(gk_ewm_dfs[1], columns=gk_columns)
tester

,team,game_id,Player,Age,Min,SoTA,GA,Saves,save_pct,PSxG,...,launch_pct,AvgLen_4,Att_5,launch_1pct,AvgLen_7,Opp,Stp,stp_pct,opa,AvgDist
0,Toulouse,2717,Alban Lafont,18.529,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toulouse,2731,Alban Lafont,18.551,90,7,3,4,57.1,2.8,...,100,58.9,9,100,69,23,1,4.3,0,14.7
2,Toulouse,2745,Alban Lafont,18.573,90,3.75,1.375,2.375,80.3375,1.28333,...,70.2083,48.6083,8.45833,86.4583,62.4458,18.125,1.54167,9.71667,0,17.4083
3,Toulouse,2748,Alban Lafont,18.589,90,7.36028,3.18014,4.57044,72.9852,2.5388,...,70.6734,48.9173,6.71824,91.7436,64.6136,15.7344,1.72055,12.4423,0,15.8829
4,Toulouse,2761,Alban Lafont,18.627,90,5.98391,2.80761,3.44339,60.4581,1.9268,...,65.7891,48.5331,5.8602,94.3499,66.7874,14.5556,1.4931,11.1347,0.315661,17.5929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Nantes,3943,Alban Lafont,21.986,90,4.51451,1.59443,3.25024,67.0678,1.69035,...,49.0543,40.7536,6.4803,78.0232,54.8389,9.57443,0.722524,6.34437,0.738872,15.1897
119,Nantes,3953,Alban Lafont,22.003,90,3.97382,1.50298,2.7502,55.4717,1.49183,...,50.9075,42.2838,6.86795,81.4043,56.8329,10.563,0.765212,6.33754,0.6252,14.3605
120,Nantes,3966,Alban Lafont,22.022,90,3.97785,1.57944,2.63478,54.5433,1.43155,...,45.5371,40.0248,6.42673,72.7267,53.1663,10.0148,0.95518,9.76254,0.682861,14.6743
121,Nantes,3976,Alban Lafont,22.03,90,4.1351,1.64414,2.53713,55.4547,1.39593,...,43.9621,39.344,6.51492,76.9226,55.7561,9.70485,0.962075,10.1837,0.731652,14.4167


In [297]:
concat_array = np.concatenate(attack_ewm_dfs2, axis=0)
att_ewm_2 = pd.DataFrame(concat_array, columns=att_columns)

In [298]:
concat_array = np.concatenate(defence_ewm_dfs2, axis=0)
def_ewm_2 = pd.DataFrame(concat_array, columns=def_columns)

In [299]:
concat_array = np.concatenate(gk_ewm_dfs, axis=0)
gk_ewm_2 = pd.DataFrame(concat_array, columns=gk_columns)

#### Now I have gathered the EWMs I need to categorise the position variables so that they are always in a similiar order for each team 

# Categorising Positions for defence and attack

In [301]:
def_ewm_2['Pos'] = pd.Categorical(def_ewm_2['Pos'], ['CB', 'RB', 'LB', 'DF', 'WB', 'DM', 'CM', 'MF', 'RM', 'LM', 'AM', 'RW', 'LW', 'FW'])

In [311]:
def_ewm_3 = def_ewm_2.sort_values(['Pos', 'Min'], ascending=[True, False])
def_ewm_3

,Player,Pos,team,game_id,Age,Min,Tkl_1,TklW,Def 3rd_2,Mid 3rd_3,...,Mid 3rd_7,Att 3rd_8,Blocks,Sh,ShSv,Pass,Int,Tkl+Int,Clr,Err
133989,Steve Birnbaum,CB,D.C. United,8639,28.107,94.6288,0.449423,0.29675,0.182225,0.264828,...,0.908586,0.310953,1.38705,1.00162,0.0106545,0.385427,0.472185,0.921608,5.63272,0.0355073
138857,Reto Ziegler,CB,FC Dallas,9049,34.121,94.6149,0.926058,0.844448,0.709289,0.163544,...,2.60908,0.257405,2.31471,1.51803,0,0.796677,0.44325,1.36931,3.6784,0.010624
135226,Jonathan Mensah,CB,Columbus Crew,9366,30.4,94.6118,0.330118,0.255907,0.172092,0.158025,...,1.25352,0.218655,2.03816,1.27189,9.19262e-07,0.766263,0.629925,0.960043,6.15714,0.0944989
133932,Jack Elliott,CB,Philadelphia Union,9040,24.164,94.6059,1.40269,0.882257,1.0327,0.265728,...,3.56389,0.498027,1.97327,0.94317,0.0342739,1.0301,0.377677,1.78037,9.25759,0.0565348
147478,David Romney,CB,Nashville SC,9363,27.466,94.5084,0.621509,0.501199,0.515759,0.073418,...,2.36932,0.304372,1.1783,0.671668,2.59692e-05,0.506629,0.212469,0.833979,5.62165,0.00232538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199147,Joel Mumbongo,FW,Burnley,1342,22.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199151,Emmanuel Dennis,FW,Koln,6850,23.211,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199157,Aleksandr Kokorin,FW,Fiorentina,5322,29.885,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199159,Petar Musa,FW,Union Berlin,6857,22.929,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [308]:
att_ewm_2['Pos'] = pd.Categorical(att_ewm_2['Pos'], ['FW', 'RW', 'LW', 'AM', 'RM', 'LM', 'CM', 'MF', 'WB', 'DM', 'RB', 'LB', 'DF', 'CB'])

In [ ]:
att_ewm_3 = att_ewm_2.sort_values('Pos')

In [312]:
def_ewm_groups = def_ewm_3.groupby(['game_id', 'team']).apply(lambda x: x.sort_values(['Pos', 'Min'], ascending=[True, False]))

In [313]:
def_ewm_groups.reset_index(drop=True, inplace=True)

In [314]:
groups = def_ewm_groups.groupby(['game_id', 'team']).head(7)

In [315]:
att_ewm_groups = att_ewm_3.groupby(['game_id', 'team']).apply(lambda x: x.sort_values(['Pos', 'Min'], ascending=[True, False])).reset_index(drop=True)

In [316]:
att_groups = att_ewm_groups.groupby(['game_id', 'team']).head(7)

In [317]:
gk_groups = gk_ewm_2.groupby(['team', 'game_id']).apply(lambda x: x.sort_values('Min')).reset_index(drop=True)

In [318]:
gk_groups = gk_groups.groupby(['game_id', 'team']).head(1)

In [450]:
test_A = att_groups.groupby(['game_id', 'team'])

test_D = groups.groupby(['game_id', 'team'])

test_G = gk_groups.groupby(['game_id', 'team'])
# 20000011 ['1610612744', '1610612756']

t3 = test_D.get_group((1364, 'Chelsea'))
t2 = test_A.get_group((1364, 'Sheffield United')).sort_values(by='Pos')
t4 = test_G.get_group((1364, 'Chelsea')).sort_values(by='Min')

def_prac = pd.DataFrame([np.array(t3).ravel()], columns=def_columns*7)
att_prac = pd.DataFrame([np.array(t2).ravel()], columns=att_columns*7)
gk_prac = pd.DataFrame([np.array(t4).ravel()], columns=gk_columns)


display(att_prac)
display(def_prac)
display(gk_prac)


NameError: name 'att_groups' is not defined

In [ ]:
def data_maker(df):
    
    groupby = df.groupby(['game_id', 'team'])
    
    full = []
    for group in groupby:
        full.append([np.array(group).ravel()])
    
    dfs = []
    for i in tqdm(range(len(full_attack))):
        dfs.append(full_attack[i][0][1])

    frames = []
    for df1 in tqdm(dfs):
        new_df = pd.DataFrame(np.array(df1).ravel())
        t = new_df.transpose()
        frames.append(t)
        
    ewms = pd.concat(frames)  
    ewms.columns = list(df.columns) * 7
    
return ewms

In [322]:
full_attack = []
for group in tqdm(test_A):
    full_attack.append([np.array(group).ravel()])

att_dfs = []
for i in tqdm(range(len(full_attack))):
    att_dfs.append(full_attack[i][0][1])
    
frames = []
for df in tqdm(att_dfs):
    new_df = pd.DataFrame(np.array(df).ravel())
    t = new_df.transpose()
    frames.append(t)
    
new_att_df = pd.concat(frames)
new_att_df.columns = att_columns * 7 

100%|██████████| 15182/15182 [04:05<00:00, 61.83it/s]


In [323]:
full_defence = []
for group in tqdm(test_D):
    full_defence.append([np.array(group).ravel()])

def_dfs = []
for i in tqdm(range(len(full_defence))):
    def_dfs.append(full_defence[i][0][1])
    
frames = []
for df in tqdm(def_dfs):
    new_df = pd.DataFrame(np.array(df).ravel())
    t = new_df.transpose()
    frames.append(t)
    
new_def_df = pd.concat(frames)
new_def_df.columns = def_columns * 7 

100%|██████████| 15182/15182 [03:45<00:00, 67.34it/s]


# Creating individual columns for clear Joining in SQL

In [324]:
new_def_df.reset_index(drop=True, inplace=True)

In [325]:
def_set = list(new_def_df.iloc[:, 0:29].columns)

In [326]:
new_cols = []
for i in range(1,8):
    for col in def_set:
        if col == 'Player':
            new_col = f'def_player{i}'
            new_cols.append(new_col)
        else:
            new_col = f'def_player{i}_{col}'
            new_cols.append(new_col)


In [327]:
new_def_df.columns = new_cols

## Now for attack

In [328]:
att_set = list(new_att_df.iloc[:, 0:32].columns)

In [329]:
new_cols = []
for i in range(1,8):
    for col in att_set:
        if col == 'Player':
            new_col = f'att_player{i}'
            new_cols.append(new_col)
        else:
            new_col = f'att_player{i}_{col}'
            new_cols.append(new_col)

In [330]:
new_att_df.columns = new_cols

In [331]:
new_att_df.reset_index(drop=True, inplace=True)

# Creating mean and sum ages for attack and defence

In [332]:
sum_ages = new_def_df['def_player1_Age'] + new_def_df['def_player2_Age'] + new_def_df['def_player3_Age'] + new_def_df['def_player4_Age'] + new_def_df['def_player5_Age'] + new_def_df['def_player6_Age'] + new_def_df['def_player7_Age']

In [333]:
mean_ages = (new_def_df['def_player1_Age'] + new_def_df['def_player2_Age'] + new_def_df['def_player3_Age'] + new_def_df['def_player4_Age'] + new_def_df['def_player5_Age'] + new_def_df['def_player6_Age'] + new_def_df['def_player7_Age'])/7

In [334]:
new_def_df['def_mean_ages'] = mean_ages

In [335]:
new_def_df['def_sum_ages'] = sum_ages

In [336]:
sum_ages = new_att_df['att_player1_Age'] + new_att_df['att_player2_Age'] + new_att_df['att_player3_Age'] + new_att_df['att_player4_Age'] + new_att_df['att_player5_Age'] + new_att_df['att_player6_Age'] + new_att_df['att_player7_Age']

In [337]:
new_att_df['att_sum_ages'] = sum_ages

In [338]:
mean_ages = (new_att_df['att_player1_Age'] + new_att_df['att_player2_Age'] + new_att_df['att_player3_Age'] + new_att_df['att_player4_Age'] + new_att_df['att_player5_Age'] + new_att_df['att_player6_Age'] + new_att_df['att_player7_Age'])/7

In [339]:
new_att_df['att_mean_ages'] = mean_ages

# Getting rid of unnescesary columns and renaming

In [340]:
droppers = [col for col in new_def_df.columns if 'Player1' not in col and 'Date' in col or 'team' in col or 'game' in col][2:]

In [341]:
new_def_df.drop(columns=[col for col in droppers], inplace=True)

In [342]:
att_droppers = [col for col in new_att_df.columns if 'Player1' not in col and 'Date' in col or 'team' in col or 'game' in col][2:]

In [343]:
new_att_df.drop(columns=[col for col in att_droppers], inplace=True)

In [344]:
new_def_df.rename(columns={'def_player1_Date': 'def_Date', 'def_player1_team': 'def_team', 'def_player1_game_id': 'def_game_id'}, inplace=True)

In [345]:
new_att_df.rename(columns={'att_player1_Date': 'att_Date', 'att_player1_team': 'att_team', 'att_player1_game_id': 'att_game_id'}, inplace=True)

# Saving as csv

In [346]:
new_att_df.to_csv('attack_ewm_final_12.csv')
new_def_df.to_csv('defence_ewm_final_12.csv')
gk_groups.to_csv('gk_ewm_final_12.csv')

#### Here I tried a few different EWMs, hence multiple saves to SQL

In [347]:
new_att_df.to_sql('att_ewm_12', engine, index=False, if_exists='replace')

In [348]:
new_def_df.to_sql('def_ewm_12', engine, index=False, if_exists='replace')

In [349]:
gk_groups.to_sql('gk_ewm_12', engine, index=False, if_exists='replace')

In [1833]:
def_practice.to_sql('def_practice', engine, index=False, if_exists='replace')
att_practice.to_sql('att_practice', engine, index=False, if_exists='replace')

#### Right joining the defence onto the attack

In [350]:
big_one_12 = q('''SELECT * FROM att_ewm_12
                RIGHT JOIN def_ewm_12
                 ON att_ewm_12.att_game_id = def_ewm_12.def_game_id
                 WHERE att_ewm_12.att_team != def_ewm_12.def_team
                 ORDER BY att_ewm_12.att_game_id
                 ''')
big_one_12

,att_player1,att_player1_Pos,att_team,att_game_id,att_player1_Age,att_player1_Min,att_player1_Gls,att_player1_Ast,att_player1_PK,att_player1_PKatt,...,def_player7_Blocks,def_player7_Sh,def_player7_ShSv,def_player7_Pass,def_player7_Int,def_player7_Tkl+Int,def_player7_Clr,def_player7_Err,def_mean_ages,def_sum_ages
0,Olivier Giroud,FW,Arsenal,1,30.863,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.785143,187.496
1,Jamie Vardy,FW,Leicester City,1,30.581,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.832714,173.829
2,Stefano Okaka,FW,Watford,2,28.008,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.146000,169.022
3,Divock Origi,FW,Liverpool,2,22.318,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.379714,198.658
4,Jay Rodriguez,FW,West Bromwich Albion,3,28.038,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.156571,197.096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15177,Wayne Rooney,FW,D.C. United,9694,33.671,85.484408,0.420342,0.243684,0.198418,0.209198,...,0.555856,0.009050,0.0,0.546807,1.294893,3.183022,1.988298,0.000000,28.468571,199.280
15178,Alejandro Pozuelo,FW,Toronto FC,9695,27.764,78.749773,0.199315,0.170297,0.023390,0.023390,...,1.772356,0.119059,0.0,1.653297,0.498380,2.356174,0.547652,0.002808,28.029286,196.205
15179,Brandon Vazquez,FW,Atlanta United,9695,20.699,10.601450,0.000000,0.000000,0.000000,0.000000,...,0.566059,0.062627,0.0,0.503433,0.500491,2.262134,0.571763,0.000000,27.342714,191.399
15180,Lucas Melano,FW,Portland Timbers,9696,26.321,16.659069,0.018448,0.021802,0.035987,0.035987,...,0.695551,0.054641,0.0,0.640911,0.341479,1.580020,0.593741,0.040877,28.517857,199.625


In [351]:
big_one_12.to_sql('big_one_12', engine, index=False, if_exists='replace')

In [1527]:
meta.to_csv('clean_meta.csv')

#### Right joining the goalkeepers onto the end of the defence and attack dataframe

In [352]:
bigger_12 = q('''SELECT * FROM big_one_12
              RIGHT JOIN gk_ewm_12
              ON big_one_12.def_game_id = gk_ewm_12.game_id
              WHERE big_one_12.def_team = gk_ewm_12.team''')
bigger_12

,att_player1,att_player1_Pos,att_team,att_game_id,att_player1_Age,att_player1_Min,att_player1_Gls,att_player1_Ast,att_player1_PK,att_player1_PKatt,...,launch_pct,AvgLen_4,Att_5,launch_1pct,AvgLen_7,Opp,Stp,stp_pct,opa,AvgDist
0,Olivier Giroud,FW,Arsenal,1,30.863,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jamie Vardy,FW,Leicester City,1,30.581,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stefano Okaka,FW,Watford,2,28.008,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Divock Origi,FW,Liverpool,2,22.318,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jay Rodriguez,FW,West Bromwich Albion,3,28.038,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15177,Wayne Rooney,FW,D.C. United,9694,33.671,85.484408,0.420342,0.243684,0.198418,0.209198,...,46.360574,42.115993,7.920129,74.280753,58.129755,7.386836,0.327130,3.116473,0.962103,15.813143
15178,Alejandro Pozuelo,FW,Toronto FC,9695,27.764,78.749773,0.199315,0.170297,0.023390,0.023390,...,43.983977,39.398200,9.276592,58.232868,45.211237,8.076139,0.985866,12.628805,0.725605,15.404620
15179,Brandon Vazquez,FW,Atlanta United,9695,20.699,10.601450,0.000000,0.000000,0.000000,0.000000,...,33.410791,33.584565,7.053628,51.061660,44.640942,7.625385,1.148179,12.561098,1.634156,17.263141
15180,Lucas Melano,FW,Portland Timbers,9696,26.321,16.659069,0.018448,0.021802,0.035987,0.035987,...,41.944737,38.015320,5.396458,64.820049,49.234787,4.523701,0.351328,8.018166,0.560575,14.911504


In [353]:
bigger_12.to_sql('bigger_12', engine, index=False, if_exists='replace')

#### Right joining the meta data onto the end

In [355]:
huge12 = q('''SELECT * FROM bigger_12 
     RIGHT JOIN meta
     ON bigger_12.att_game_id = meta."Game_id"''')
huge12

,att_player1,att_player1_Pos,att_team,att_game_id,att_player1_Age,att_player1_Min,att_player1_Gls,att_player1_Ast,att_player1_PK,att_player1_PKatt,...,Attendance,home_captain,Home_team,Away_team,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG
0,Olivier Giroud,FW,Arsenal,1,30.863,NaN,NaN,NaN,NaN,NaN,...,59387.0,Petr Cech,Arsenal,Leicester City,1.0,Wes Morgan,3.0,4.0,2.1,1.6
1,Jamie Vardy,FW,Leicester City,1,30.581,NaN,NaN,NaN,NaN,NaN,...,59387.0,Petr Cech,Arsenal,Leicester City,1.0,Wes Morgan,3.0,4.0,2.1,1.6
2,Stefano Okaka,FW,Watford,2,28.008,NaN,NaN,NaN,NaN,NaN,...,20407.0,Heurelho Gomes,Watford,Liverpool,2.0,Jordan Henderson,3.0,3.0,2.0,3.0
3,Divock Origi,FW,Liverpool,2,22.318,NaN,NaN,NaN,NaN,NaN,...,20407.0,Heurelho Gomes,Watford,Liverpool,2.0,Jordan Henderson,3.0,3.0,2.0,3.0
4,Jay Rodriguez,FW,West Bromwich Albion,3,28.038,NaN,NaN,NaN,NaN,NaN,...,25011.0,Jake Livermore,West Bromwich Albion,Bournemouth,3.0,Simon Francis,0.0,1.0,1.2,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15177,Wayne Rooney,FW,D.C. United,9694,33.671,85.484408,0.420342,0.243684,0.198418,0.209198,...,17398.0,Wayne Rooney,D.C. United,Orlando City,9694.0,Nani,0.0,1.0,0.7,1.4
15178,Alejandro Pozuelo,FW,Toronto FC,9695,27.764,78.749773,0.199315,0.170297,0.023390,0.023390,...,24598.0,Quentin Westberg,Toronto FC,Atlanta United,9695.0,Michael Parkhurst,2.0,3.0,2.1,2.5
15179,Brandon Vazquez,FW,Atlanta United,9695,20.699,10.601450,0.000000,0.000000,0.000000,0.000000,...,24598.0,Quentin Westberg,Toronto FC,Atlanta United,9695.0,Michael Parkhurst,2.0,3.0,2.1,2.5
15180,Lucas Melano,FW,Portland Timbers,9696,26.321,16.659069,0.018448,0.021802,0.035987,0.035987,...,15167.0,Saphir Taider,Montreal Impact,Portland Timbers,9696.0,Jeff Attinella,1.0,2.0,1.0,1.5


In [356]:
huge12.to_sql('huge12', engine, index=False, if_exists='replace')

In [7]:
huge = q('''SELECT * FROM huge''')

In [357]:
def_home = []
for x, y in zip(huge12.Home_team, huge12.def_team):
    if x == y:
        def_home.append(1)
    else:
        def_home.append(0)

In [358]:
huge12['def_home']= def_home

#### Creating my Y variable

In [359]:
clean_sheets = []
for x, y, z in zip(huge12.def_home, huge12.home_conceded, huge12.away_conceded):
    if x ==0 and z>0:
        clean_sheets.append(0)
    elif x==0 and z==0:
        clean_sheets.append(1)
    elif x==1 and y>0:
        clean_sheets.append(0)
    else:
        clean_sheets.append(1)
    

In [358]:
huge['clean_sheets'] = clean_sheets

In [360]:
huge12['clean_sheets'] = clean_sheets

#### Engineering home and away managers in order to dummify

In [362]:
def_manager = []
att_manager = []
for h_manager, a_manager, def_home in zip(huge12.Home_manager, huge12.Away_manager, huge12.def_home):
    if def_home == 0:
        att_manager.append(h_manager)
        def_manager.append(a_manager)
    elif def_home ==1:
        def_manager.append(h_manager)
        att_manager.append(a_manager)

In [363]:
huge12['att_manager'] = att_manager
huge12['def_manager'] = def_manager

In [ ]:
huge12.to_sql('huge12', engine, index=False, if_exists='replace')

In [9]:
homes = huge.groupby('def_home')

In [11]:
home = homes.get_group(1)

In [150]:
huge

,att_Date,att_team,att_game_id,att_player1,att_player1_Pos,att_player1_Age,att_player1_Min,att_player1_Gls,att_player1_Ast,att_player1_PK,...,Game_id,away_captain,home_conceded,away_conceded,home_xG,away_xG,def_home,clean_sheets,att_manager,def_manager
0,2017-08-11,Arsenal,1,Olivier Giroud,FW,30.863,NaN,NaN,NaN,NaN,...,1.0,Wes Morgan,3.0,4.0,2.1,1.6,0,0,Arsene Wenger,Craig Shakespeare
1,2017-08-11,Leicester City,1,Jamie Vardy,FW,30.581,NaN,NaN,NaN,NaN,...,1.0,Wes Morgan,3.0,4.0,2.1,1.6,1,0,Craig Shakespeare,Arsene Wenger
2,2017-08-12,Liverpool,2,Divock Origi,FW,22.318,NaN,NaN,NaN,NaN,...,2.0,Jordan Henderson,3.0,3.0,2.0,3.0,1,0,Jurgen Klopp,Marco Silva
3,2017-08-12,Watford,2,Stefano Okaka,FW,28.008,NaN,NaN,NaN,NaN,...,2.0,Jordan Henderson,3.0,3.0,2.0,3.0,0,0,Marco Silva,Jurgen Klopp
4,2017-08-12,Bournemouth,3,Jermain Defoe,FW,34.847,NaN,NaN,NaN,NaN,...,3.0,Simon Francis,0.0,1.0,1.2,0.4,1,1,Eddie Howe,Tony Pulis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15177,2019-06-26,Orlando City,9694,Santiago Patino,FW,22.296,15.955011,0.079275,0.014245,0.000000,...,9694.0,Nani,0.0,1.0,0.7,1.4,1,1,James O'Connor,Ben Olsen
15178,2019-06-26,Atlanta United,9695,Brandon Vazquez,FW,20.699,45.543617,0.121933,0.189528,0.000807,...,9695.0,Michael Parkhurst,2.0,3.0,2.1,2.5,1,0,Frank de Boer,Greg Vanney
15179,2019-06-26,Toronto FC,9695,Alejandro Pozuelo,FW,27.764,93.654518,0.289489,0.062272,0.206455,...,9695.0,Michael Parkhurst,2.0,3.0,2.1,2.5,0,0,Greg Vanney,Frank de Boer
15180,2019-06-26,Montreal Impact,9696,Maximiliano Urruti,FW,28.340,49.839910,0.231515,0.001722,0.000162,...,9696.0,Jeff Attinella,1.0,2.0,1.0,1.5,0,0,Remi Garde,Giovanni Savarese
